# Using Redis and OpenAI to chat with PDF documents

This notebook demonstrates how to use RedisAI and (Azure) OpenAI to chat with PDF documents. The PDF included is
a informational brochure about the Chevy Colorado pickup truck.

In this notebook, we will use LLamaIndex to chunk, vectorize, and store the PDF document in Redis as vectors
alongside associated text. The query interface provided by LLamaIndex will be used to search for relevant
information given queries from the user.

In [ ]:
# Install the requirements
%pip install redis PyPDF2 python-dotenv transformers tiktoken

In [ ]:
%pip install git+https://github.com/jerryjliu/llama_index@main

In [1]:
import os
import textwrap
import openai
from langchain.llms import AzureOpenAI, OpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index.vector_stores import RedisVectorStore
from llama_index import LangchainEmbedding
from llama_index import (
    GPTVectorStoreIndex,
    SimpleDirectoryReader,
    LLMPredictor,
    PromptHelper,
    ServiceContext,
    StorageContext
)
import sys

import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO) # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


/Users/sam.partee/.virtualenvs/llama3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load the .env file in the parent directory into the current environment
from dotenv import load_dotenv
load_dotenv('./.env')

True

# Azure OpenAI and OpenAI

The notebook allows the user two choose between using the OpenAI and Azure OpenAI endpoints. Make sure to follow the instructions in the README and set the .env correctly according to whichever API you are using. 

NOTE: ONLY ONE API CAN BE USED AT A TIME.

## Azure OpenAI 

Here we setup the AzureOpenAI models and API keys that we set by reading from the environment above. The ``PromptHelper`` sets the parameters for the OpenAI model. The classes defined here are used together to provide a QnA interface between the user and the LLM.

In [3]:
# setup Llama Index to use Azure OpenAI
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OPENAI_API_BASE")
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

# Get the OpenAI model names ex. "text-embedding-ada-002"
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
text_model = os.getenv("OPENAI_TEXT_MODEL")


print(f"Using models: {embedding_model} and {text_model}")

# get the Azure Deployment name for the model
embedding_model_deployment = os.getenv("AZURE_EMBED_MODEL_DEPLOYMENT_NAME")
text_model_deployment = os.getenv("AZURE_TEXT_MODEL_DEPLOYMENT_NAME")

print(f"Using deployments: {embedding_model_deployment} and {text_model_deployment}")


Using models: text-embedding-ada-002 and text-davinci-003
Using deployments: embed and textgen


In [4]:

llm = AzureOpenAI(deployment_name=text_model_deployment, model_kwargs={
    "api_key": openai.api_key,
    "api_base": openai.api_base,
    "api_type": openai.api_type,
    "api_version": openai.api_version,
})
llm_predictor = LLMPredictor(llm=llm)

embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model=embedding_model,
        deployment=embedding_model_deployment,
        openai_api_key= openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


## OpenAI

The ``OpenAI`` class provides a simple interface to the OpenAI API.


In [5]:
# setup Llama Index to use Azure OpenAI
openai.api_type = "openai"
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_API_KEY")

# Get the OpenAI model names ex. "text-embedding-ada-002"
embedding_model = os.getenv("OPENAI_EMBEDDING_MODEL")
text_model = os.getenv("OPENAI_TEXT_MODEL")


print(f"Using models: {embedding_model} and {text_model}")

Using models: text-embedding-ada-002 and text-davinci-003


In [6]:

llm = OpenAI(model_kwargs={
    "api_key": openai.api_key,
    "api_base": openai.api_base,
    "api_type": openai.api_type,
    "api_version" : openai.api_version,

})
llm_predictor = LLMPredictor(llm=llm)

embedding_llm = LangchainEmbedding(
    OpenAIEmbeddings(
        model=embedding_model,
        openai_api_version=openai.api_version,
        openai_api_key= openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
    ),
    embed_batch_size=1,
)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


### LLamaIndex

[LlamaIndex](https://github.com/jerryjliu/llama_index) (GPT Index) is a project that provides a central interface to connect your LLM's with external data sources. It provides a simple interface to vectorize and store embeddings in Redis, create search indices using Redis, and perform vector search to find context for generative models like GPT.

Here we will use it to load in the documents (Chevy Colorado Brochure).

In [5]:
# load documents
documents = SimpleDirectoryReader('./docs').load_data()
print('Document ID:', documents[0].doc_id)

Document ID: 77fb1707-6c1d-48f9-b76b-14d48cfe0156


Llamaindex also works with frameworks like langchain to make prompting and other aspects of a chat based application easier. Here we can use the ``PromptHelper`` class to help us generate prompts for the (Azure) OpenAI model. The will be off by default as it can be tricky to setup correctly.

In [6]:
# set number of output tokens
num_output = int(os.getenv("OPENAI_MAX_TOKENS"))
# max LLM token input size
max_input_size = int(os.getenv("CHUNK_SIZE"))
# set maximum chunk overlap
max_chunk_overlap = int(os.getenv("CHUNK_OVERLAP"))

prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [7]:

# define the service we will use to answer questions
# if you executive the Azure OpenAI code above, your Azure Models and creds will be used and the same for OpenAI
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor,
    embed_model=embedding_llm,
#    prompt_helper=prompt_helper # uncomment to use prompt_helper.
)

## Initialize Redis as a Vector Database

Now we have our documents read in, we can initialize the ``RedisVectorStore``. This will allow us to store our vectors in Redis and create an index.

The ``GPTVectorStoreIndex`` will then create the embeddings from the text chunks by calling out to OpenAI's API. The embeddings will be stored in Redis and an index will be created.

NOTE: If you didn't set the ``OPENAI_API_KEY`` environment variable, you will get an error here.

In [8]:
# get the Redis address from the environment.

# If using SSL, use rediss:// (this is required when using ACRE)
if bool(os.getenv("REDIS_SSL")):
    redis_address = f'rediss://default:{os.getenv("REDIS_PASSWORD")}@{os.getenv("REDIS_ADDRESS")}'

# If using a password but no SSL, use the format redis://:password@address:port
elif os.getenv("REDIS_PASSWORD") is not None:
    redis_address = f'redis://default:{os.getenv("REDIS_PASSWORD")}@{os.getenv("REDIS_ADDRESS")}:{os.getenv("REDIS_PORT")}'

# for local unsecured Redis, use redis://address:port
else:
    redis_address = f'redis://{os.getenv("REDIS_ADDRESS")}:{os.getenv("REDIS_PORT")}'

print(f"Using Redis address: {redis_address}")
vector_store = RedisVectorStore(
    index_name="chevy_docs",
    index_prefix="blog",
    redis_url=redis_address,
    overwrite=True
)

# access the underlying client in the RedisVectorStore implementation to ping the redis instance
vector_store.client.ping()

Using Redis address: redis://localhost:6379


True

In [9]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = GPTVectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    service_context=service_context
)

INFO:llama_index.vector_stores.redis:Deleting index chevy_docs
Deleting index chevy_docs
INFO:llama_index.vector_stores.redis:Creating index chevy_docs
Creating index chevy_docs
INFO:llama_index.vector_stores.redis:Added 19 documents to index chevy_docs
Added 19 documents to index chevy_docs
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 17067 tokens
> [build_index_from_nodes] Total embedding token usage: 17067 tokens


## Start Querying information from the Document

Now that we have our document stored in the index, we can ask questions against the index. The index will use the data stored in itself as the knowledge base for chatgpt.

In [10]:
query_engine = index.as_query_engine()
response = query_engine.query("What types of variants are available for the Chevrolet Colorado?")
print("\n", textwrap.fill(str(response), 100))

INFO:llama_index.vector_stores.redis:Querying index chevy_docs
Querying index chevy_docs
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['blog_cd511123-5d91-44ef-98c7-0c813f2f71fb', 'blog_985903b8-474e-4bbc-99bd-2ea9680a082f']
Found 2 results for query with id ['blog_cd511123-5d91-44ef-98c7-0c813f2f71fb', 'blog_985903b8-474e-4bbc-99bd-2ea9680a082f']
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 11 tokens
> [retrieve] Total embedding token usage: 11 tokens


Token indices sequence length is longer than the specified maximum sequence length for this model (1930 > 1024). Running this sequence through the model will result in indexing errors


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2066 tokens
> [get_response] Total LLM token usage: 2066 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens

  The Chevrolet Colorado is available in four models: WT, LT, Z71, and ZR2. The WT and LT models are
available with a 2.5L 4-cylinder or 3.6L V6 engine, while the Z71 and ZR2 models are available with
a 3.6L V6 or Duramax 2.8L Turbo-Diesel engine. The Colorado is available in both Extended Cab and
Crew Cab configurations, with the Crew Cab Short Box and Crew Cab Long Box being the most popular.
The ZR2 Bison Edition, ZR2 Dusk Special Edition, and ZR2 Midnight Special Edition are also
available.


In [11]:
response = query_engine.query("What is the maximum towing capacity of the chevy colorado?")
print("\n", textwrap.fill(str(response), 100))

INFO:llama_index.vector_stores.redis:Querying index chevy_docs
Querying index chevy_docs
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['blog_cd511123-5d91-44ef-98c7-0c813f2f71fb', 'blog_0e87353a-b93b-48fa-995b-1f83c1994d2f']
Found 2 results for query with id ['blog_cd511123-5d91-44ef-98c7-0c813f2f71fb', 'blog_0e87353a-b93b-48fa-995b-1f83c1994d2f']
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 14 tokens
> [retrieve] Total embedding token usage: 14 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1912 tokens
> [get_response] Total LLM token usage: 1912 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens

  The maximum towing capacity of th

In [12]:
response = query_engine.query("What are the main differences between the three engine types available for the Chevy Colorado?")
print("\n", textwrap.fill(str(response), 100))

INFO:llama_index.vector_stores.redis:Querying index chevy_docs
Querying index chevy_docs
INFO:llama_index.vector_stores.redis:Found 2 results for query with id ['blog_cd511123-5d91-44ef-98c7-0c813f2f71fb', 'blog_985903b8-474e-4bbc-99bd-2ea9680a082f']
Found 2 results for query with id ['blog_cd511123-5d91-44ef-98c7-0c813f2f71fb', 'blog_985903b8-474e-4bbc-99bd-2ea9680a082f']
INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 16 tokens
> [retrieve] Total embedding token usage: 16 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2074 tokens
> [get_response] Total LLM token usage: 2074 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens

  The main differences between the 